In [1]:
import os
import pprint

import keras
import numpy as np
import sklearn.metrics
import tensorflow as tf

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import average, AveragePooling2D, concatenate, Conv2D, Conv3D, Dense, Flatten, Input, Reshape
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold

PATCH_HEIGHT = 28
PATCH_WIDTH = 28

data_dir = 'data'
if not os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
if not os.path.exists('logs'):
    os.mkdir('logs')

pp = pprint.PrettyPrinter(indent=4)

Using TensorFlow backend.


In [2]:
ct_train = np.load(os.path.join(data_dir, 'ct_train.npy'))
pet_train = np.load(os.path.join(data_dir, 'pet_train.npy'))
y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
y_train_targets = np.argmax(y_train, axis=1)

ct_test = np.load(os.path.join(data_dir, 'ct_test.npy'))
pet_test = np.load(os.path.join(data_dir, 'pet_test.npy'))
y_test = np.load(os.path.join(data_dir, 'y_test.npy'))

def get_train(mode=None, indices=None):
    if mode == 'ct':
        return ct_train[indices] if indices is not None else ct_train
    elif mode == 'pet':
        return pet_train[indices] if indices is not None else pet_train
    else:
        return [ct_train[indices], pet_train[indices]] if indices is not None else [ct_train, pet_train]

def get_test(mode=None):
    if mode == 'ct':
        return ct_test
    elif mode == 'pet':
        return pet_test
    else:
        return [ct_test, pet_test]

In [3]:
def confusion_matrix(y_true, y_pred):
    y_true_targets = np.argmax(y_true, axis=1)
    y_pred_targets = np.argmax(y_pred, axis=1)
    return sklearn.metrics.confusion_matrix(y_true_targets, y_pred_targets)

def accuracy(y_true, y_pred):
    y_true_targets = np.argmax(y_true, axis=1)
    y_pred_targets = np.argmax(y_pred, axis=1)
    return sklearn.metrics.accuracy_score(y_true_targets, y_pred_targets)

def f1(y_true, y_pred):
    c_matrix = confusion_matrix(y_true, y_pred)
    tp = c_matrix[1][1]
    fp = c_matrix[0][1]
    fn = c_matrix[1][0]
    return 2 * tp / (2 * tp + fn + fp)

In [8]:
def train_model(model_fn, name, batch_size=32, epochs=8, patience=2, mode=None, save=False, n_splits=5, val=True):
    print('Train...')

    best_model_path = os.path.join('checkpoints', f'best_model_{name}.h5')
    log_dir = os.path.join('logs', f'{name}')

    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    callbacks = []
    
    if val:
        callbacks.append(EarlyStopping(monitor='val_acc', patience=patience))
    
    if save:
        callbacks.append(ModelCheckpoint(best_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True))
        callbacks.append(TensorBoard(log_dir=log_dir, histogram_freq=1, batch_size=batch_size, write_graph=False, write_grads=True, write_images=True))
    
    if n_splits > 1:
        cv = StratifiedKFold(n_splits=n_splits)
        split_num = 1
        preds = []
        for train, test in cv.split(ct_train, y_train_targets):
            print(f'Fold {split_num}/{n_splits}')
            model = model_fn()
            model.fit(get_train(mode, indices=train),
                      y_train[train],
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(get_train(mode, indices=test), y_train[test]),
                      verbose=1,
                      shuffle=True,
                      callbacks=callbacks)
            y_preds = model.predict(get_test(mode))
            preds.append(y_preds)
            split_num += 1

        preds = np.argmax(np.mean(preds, axis=0), axis=1)
        with tf.Session() as sess:
            preds = sess.run(tf.one_hot(preds, 2))
    else:
        model = model_fn()
        model.fit(get_train(mode),
                  y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=0.1 if val else 0.0,
                  verbose=1,
                  shuffle=True,
                  callbacks=callbacks)
        preds = model.predict(get_test(mode))
    
    acc_score = accuracy(y_test, preds)
    f1_score = f1(y_test, preds)
    print(f'F1: {f1_score}')
    print(f'Acc: {acc_score}')
    print('\n\n')
    return f1_score, acc_score
    

def train_n_sessions(model_fn, name, n, mode=None, **kwargs):
    f1s = []
    accs = []
    
    for i in range(n):
        print(f'Round {i + 1} out of {n}')
        print('-' * 101)
        f1, acc = train_model(model_fn, name, mode=mode, **kwargs)
        f1s.append(f1)
        accs.append(acc)
    
    return f1s, accs

# Type 1: Feature-Level Fusion

In [9]:
def get_type_1_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    x = Conv3D(16, (2, 2, 2), activation='relu')(x)
    x = Reshape((27, 27, 16))(x)
    x = Conv2D(36, (2, 2), activation='relu')(x)
    x = Conv2D(64, (2, 2), activation='relu')(x)
    x = Conv2D(144, (2, 2), activation='relu')(x)
    x = AveragePooling2D((23, 23))(x)
    x = Flatten()(x)
    x = Dense(864, activation='relu')(x)
    x = Dense(288, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()

    return model

In [11]:
f1s, accs = train_n_sessions(get_type_1_model, 'type_I', 10, epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1238 - acc: 0.9569    
Epoch 2/5
41338/41338 [==============================] - 22s - loss: 0.0947 - acc: 0.9669    
Epoch 3/5
41338/41338 [==============================] - 22s - loss: 0.0659 - acc: 0.9789    
Epoch 4/5
41338/41338 [==============================] - 23s - loss: 0.0566 - acc: 0.9828    
Epoch 5/5
41338/41338 [==============================] - 22s - loss: 0.0514 - acc: 0.9842    
F1: 0.9354916646532979
Acc: 0.9366096866096866



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1260 - acc: 0.9557    
Epoch 2/5
41338/41338 [==============================] - 22s - loss: 0.0920 - acc: 0.9676    
Epoch 3/5
41338/41338 [===========

In [12]:
pp.pprint(f1s)
pp.pprint(accs)

[   0.93549166465329792,
    0.8441866596361719,
    0.88781234064252934,
    0.93167106209542072,
    0.9221789883268483,
    0.84168443496801704,
    0.91638141809290952,
    0.82528486163863268,
    0.86131386861313863,
    0.87608363080061191]
[   0.93660968660968658,
    0.8596866096866097,
    0.89553656220322886,
    0.93233618233618232,
    0.92402659069325732,
    0.85897435897435892,
    0.91880341880341876,
    0.84710351377018045,
    0.87369420702754041,
    0.88461538461538458]


# Type 2: Classifier-Level Fusion

In [13]:
def get_type_2_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    ct_model = Conv2D(16, (2, 2), activation='relu')(ct_input)
    ct_model = Conv2D(36, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(64, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(144, (2, 2), activation='relu')(ct_model)
    ct_model = AveragePooling2D((23, 23))(ct_model)
    ct_model = Flatten()(ct_model)

    pet_model = Conv2D(16, (2, 2), activation='relu')(pet_input)
    pet_model = Conv2D(36, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(64, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(144, (2, 2), activation='relu')(pet_model)
    pet_model = AveragePooling2D((23, 23))(pet_model)
    pet_model = Flatten()(pet_model)

    x = concatenate([ct_model, pet_model])
    x = Dense(864, activation='relu')(x)
    x = Dense(288, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    
    return model

In [24]:
f1s_2, accs_2 = train_n_sessions(get_type_2_model, 'type_II', 10, epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 39s - loss: 0.1125 - acc: 0.9608    
Epoch 2/5
41338/41338 [==============================] - 39s - loss: 0.0780 - acc: 0.9742    
Epoch 3/5
41338/41338 [==============================] - 39s - loss: 0.0612 - acc: 0.9808    
Epoch 4/5
41338/41338 [==============================] - 40s - loss: 0.0563 - acc: 0.9831    
Epoch 5/5
41338/41338 [==============================] - 39s - loss: 0.0462 - acc: 0.9861    
F1: 0.9399090256164712
Acc: 0.9404083570750238



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 40s - loss: 0.1207 - acc: 0.9572    
Epoch 2/5
41338/41338 [==============================] - 40s - loss: 0.0799 - acc: 0.9723    
Epoch 3/5
41338/41338 [===========

In [25]:
pp.pprint(f1s_2)
pp.pprint(accs_2)

[   0.93990902561647116,
    0.9420186113099499,
    0.87112731059619886,
    0.91017666086091065,
    0.94694457603031734,
    0.8998988877654196,
    0.85527348861554564,
    0.92003900536323746,
    0.89304677623261697,
    0.91418563922942209]
[   0.94040835707502379,
    0.94230769230769229,
    0.88247863247863245,
    0.91429249762583098,
    0.94681861348528018,
    0.90598290598290598,
    0.86870845204178537,
    0.92212725546058882,
    0.8995726495726496,
    0.91856600189933524]


# Type 3: Decision-Level Fusion

In [16]:
def get_type_3_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    ct_model = Conv2D(16, (2, 2), activation='relu')(ct_input)
    ct_model = Conv2D(36, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(64, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(144, (2, 2), activation='relu')(ct_model)
    ct_model = AveragePooling2D((23, 23))(ct_model)
    ct_model = Flatten()(ct_model)
    ct_model = Dense(864, activation='relu')(ct_model)
    ct_model = Dense(288, activation='relu')(ct_model)
    ct_model = Dense(2, activation='softmax')(ct_model)

    pet_model = Conv2D(16, (2, 2), activation='relu')(pet_input)
    pet_model = Conv2D(36, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(64, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(144, (2, 2), activation='relu')(pet_model)
    pet_model = AveragePooling2D((23, 23))(pet_model)
    pet_model = Flatten()(pet_model)
    pet_model = Dense(864, activation='relu')(pet_model)
    pet_model = Dense(288, activation='relu')(pet_model)
    pet_model = Dense(2, activation='softmax')(pet_model)

    predictions = average([ct_model, pet_model])

    model = Model(inputs=[ct_input, pet_input], outputs=predictions)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    
    return model

In [17]:
f1s_3, accs_3 = train_n_sessions(get_type_3_model, 'type_III', 10, epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 42s - loss: 0.1462 - acc: 0.9541    
Epoch 2/5
41338/41338 [==============================] - 42s - loss: 0.1096 - acc: 0.9665    
Epoch 3/5
41338/41338 [==============================] - 42s - loss: 0.0949 - acc: 0.9713    - ETA: 0s - loss: 0.0950 - acc: 0.9 - ETA: 0s - loss: 0.0950 - acc: 0.
Epoch 4/5
41338/41338 [==============================] - 42s - loss: 0.0882 - acc: 0.9727    
Epoch 5/5
41338/41338 [==============================] - 42s - loss: 0.0818 - acc: 0.9741    
F1: 0.7917127071823205
Acc: 0.8209876543209876



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 42s - loss: 0.1446 - acc: 0.9549    - ETA: 0s - loss: 0.144
Epoch 2/5
41338/41338 [============

In [18]:
pp.pprint(f1s_3)
pp.pprint(accs_3)

[   0.79171270718232045,
    0.79533462926964738,
    0.81718708299973308,
    0.8242880171950564,
    0.78124122437517551,
    0.82679476914865224,
    0.84119041348432977,
    0.81365113759479957,
    0.83452855245683932,
    0.78153153153153154]
[   0.82098765432098764,
    0.82502374169040837,
    0.837369420702754,
    0.84472934472934469,
    0.8150522317188984,
    0.84591642924976262,
    0.85683760683760679,
    0.83665716999050332,
    0.85208926875593538,
    0.81576448243114907]


# Baseline: Single-Modality CNNs

In [19]:
def get_single_modality_model(summary=False):
    print('Build model...')

    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation='relu', input_shape=(PATCH_HEIGHT, PATCH_WIDTH, 1)))
    model.add(Conv2D(36, (2, 2), activation='relu'))
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(Conv2D(144, (2, 2), activation='relu'))
    model.add(AveragePooling2D((23, 23)))
    model.add(Flatten())
    model.add(Dense(864, activation='relu'))
    model.add(Dense(288, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()

    print('Model built.')
    
    return model

In [27]:
f1s_c, accs_c = train_n_sessions(get_single_modality_model, 'ct', 10, mode='ct', epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.2200 - acc: 0.9154    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.1513 - acc: 0.9465    
Epoch 3/5
41338/41338 [==============================] - 22s - loss: 0.1222 - acc: 0.9584    
Epoch 4/5
41338/41338 [==============================] - 22s - loss: 0.1024 - acc: 0.9657    
Epoch 5/5
41338/41338 [==============================] - 22s - loss: 0.0898 - acc: 0.9698    
F1: 0.5900601456157012
Acc: 0.6925451092117759



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.2110 - acc: 0.9154    
Epoch 2/5
41338/41338 [==============================] - 22s - loss: 0.1

In [28]:
pp.pprint(f1s_c)
pp.pprint(accs_c)

[   0.59006014561570119,
    0.63460967646185806,
    0.71331828442437928,
    0.52165418430478672,
    0.50863528614967835,
    0.5850556438791733,
    0.71576086956521734,
    0.61976139492199445,
    0.33049535603715169,
    0.53751250416805607]
[   0.69254510921177592,
    0.70773979107312446,
    0.75878442545109215,
    0.65123456790123457,
    0.65550807217473883,
    0.69017094017094016,
    0.75166191832858498,
    0.70489078822412155,
    0.58926875593542261,
    0.67070275403608737]


In [22]:
f1s_p, accs_p = train_n_sessions(get_single_modality_model, 'pet', 10, mode='pet', epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1216 - acc: 0.9581    
Epoch 2/5
41338/41338 [==============================] - 22s - loss: 0.0878 - acc: 0.9692    
Epoch 3/5
41338/41338 [==============================] - 22s - loss: 0.0777 - acc: 0.9745    
Epoch 4/5
41338/41338 [==============================] - 22s - loss: 0.0683 - acc: 0.9776    
Epoch 5/5
41338/41338 [==============================] - 23s - loss: 0.0606 - acc: 0.9805    
F1: 0.8876404494382022
Acc: 0.8955365622032289



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1292 - acc: 0.9555    
Epoch 2/5
41338/41338 [==============================] - 22s - loss: 0.0

In [23]:
pp.pprint(f1s_p)
pp.pprint(accs_p)

[   0.88764044943820219,
    0.90783637259733863,
    0.88583078491335376,
    0.89964699949571358,
    0.89710691823899369,
    0.91501976284584985,
    0.88877730354004514,
    0.91879921259842523,
    0.89646527951867638,
    0.92147634381966803]
[   0.89553656220322886,
    0.91120607787274455,
    0.89363722697056025,
    0.90550807217473883,
    0.90289648622981955,
    0.91832858499525161,
    0.89482431149097819,
    0.92165242165242167,
    0.90194681861348525,
    0.92473884140550811]


In [ ]:
def get_two_path_cascade(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(64, (7, 7), activation='relu')(x)
    pool1_local = AveragePooling2D((23, 23))(conv1_local)
    
    conv2_local = Conv2D(64, (3, 3), activation='relu')(pool1_local)
    pool2_local = AveragePooling2D((23, 23))(conv2_local)

    conv1_global= Conv2D(160,(13,13), activation= 'relu')(x)
    
    combine = merge([pool2_local,conv1_global], mode= 'concat', concat_axis=1)
    conv1_combine = Conv2D(5, (21,21), activation='relu')(combine)
    output = Flatten()(conv1_combine)
    
    output = Dense(2, activation='softmax')(output)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    return model


In [ ]:
c_matrices_two, accuracies_two = train_n_sessions(get_two_path_cascade, 'Casc-CNN-two', 10)

In [ ]:
def get_local_path_cascade(summary=False):
    K.clear_session()
    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(64, (7, 7), activation='relu')(x)
    pool1_local = AveragePooling2D((23, 23))(conv1_local)
    
    conv2_local = Conv2D(64, (3, 3), activation='relu')(pool1_local)
    pool2_local = AveragePooling2D((23,23))(conv2_local)
    
    conv1_combine = Conv2D(5, (21,21), activation= 'relu')(pool2_local)
    
    output = Flatten()(conv1_combine)
    output = Dense(2, activation= 'softmax')(output)
    
    model = Model(inputs=[ct_input, pet_input], outputs=output)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    if summary:
        model.summary()
        
    return model
    
    

In [ ]:
c_matrices_local, accuracies_local = train_n_sessions(get_local_path_cascade, 'Casc-CNN-local', 10)

In [ ]:
def get_global_path_cascade(summary=False):
    K.clear_session()
    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(160, (13, 13), activation='relu')(x)
    conv1_combine = Conv2D(5, (21,21) ,activation= 'relu')(conv1_local)
    
    output = Flatten()(conv1_combine)
    output = Dense(2, activation= 'softmax')(output)
    
    model = Model(inputs=[ct_input, pet_input], outputs=output)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    if summary:
        model.summary()
        
    return model

In [ ]:
c_matrices_global, accuracies_global = train_n_sessions(get_local_path_cascade, 'Casc-CNN-local', 10)